In [ ]:
import pdfplumber
import csv

In [ ]:
pdf = pdfplumber.open("mttq.pdf")

In [ ]:
import re, json
from collections import defaultdict

with open('mttq.csv', 'w', newline='') as csvfile:
    fieldnames = ['date', 'transaction_code', 'amount', 'transaction_detail',
                    # 'test'
                 ]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    for page in pdf.pages:
        start = False
        i = 0
        info = defaultdict(str)
        
        transactions = []
        date = None
        
        text_chunks = page.extract_text().split('\n')
        for text_chunk in text_chunks:
            if text_chunk == 'Postal address: Telex : (0805) 411504 VCB - VT':
                break
            if not start and text_chunk == 'Số CT/ Doc No':
                start = True
            elif start:
                match = re.fullmatch('[0-9]{2}\/09\/2024', text_chunk)
                if match:
                    i = 0
                    if info['transaction_code']:
                        info['date'] = date
                        # info['test'] = json.dumps(info['transaction_detail'].split('.'))
                        transactions.append(info)
                        info = defaultdict(str)
                    date = match.group(0)
                elif i == 0:
                    amount, first_transaction_detail_line = text_chunk.split(' ', 1)
                    info['amount'] = amount
                    info['transaction_detail'] = first_transaction_detail_line
                    match1 = re.match('(\d{1,3}(\.\d{3})*) ', first_transaction_detail_line)
                    if match1:
                        print(match1.group(1))
                        info['transaction_detail'] = info['transaction_detail'].split(' ', 1)[1]
                    i += 1
                elif i == 1:
                    info['transaction_code'] = text_chunk
                    i += 1
                else:
                    info['transaction_detail'] += ' ' + text_chunk

        if info['transaction_code']:
            info['date'] = date
            # info['test'] = json.dumps(info['transaction_detail'].split('.'))
            transactions.append(info)

        # print(page, len(transactions))
        for row in transactions:
            writer.writerow(row)